In [56]:
%reload_ext autoreload
%autoreload 2
## sys package
import os, sys
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="7"  # specify which GPU(s) to be used
sys.path.append("./prediction_models/input/prostate-cancer-grade-assessment/")
## warning off
import warnings
warnings.filterwarnings("ignore")

## general package
import random
from tqdm import tqdm_notebook as tqdm
import numpy as np
import pandas as pd
import torch
from torch.utils.data import *
from sklearn.metrics import cohen_kappa_score, roc_auc_score, roc_curve
from sklearn.metrics import confusion_matrix
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
import pickle
# device = torch.device("cuda:4" if torch.cuda.is_available() else "cpu") 
# print(device)

## customized package
from input.inputPipeline_stiching import PandaPatchDataset, crossValDataloader, data_transform
from model.resnext_ssl_extractor import Model_Infer as Model

In [57]:
data_dir = './input/panda-36x256x256-tiles-data-opt/train'
csv_file = './input/csv_pkl_files/4_fold_whole_train_wo_sus.csv'
# SAMPLE = './input/prostate-cancer-grade-assessment/sample_submission.csv'

In [58]:
models = []
weights = [f'./train/weights/Resnext50_36patch_br_deter_whole/Resnext50_36patch_br_deter_whole_{i}_best.pth.tar' for i in [3]]
for path in weights:
#     state_dict = torch.load(path,map_location=torch.device('cpu'))
    state_dict = torch.load(path)
    if "enc.0.weight" in state_dict:
        updated_state_dict = state_dict
    else:
        updated_state_dict = {}
        for key in state_dict:
            updated_state_dict[key[7:]] = state_dict[key]
    
    model = Model()
    model_dict = model.state_dict()
    updated_state_dict = {k: v for k, v in updated_state_dict.items() if k in model_dict}
    model_dict.update(updated_state_dict)
    model.load_state_dict(updated_state_dict)
    model.float()
    model.eval()
    model.cuda()
    models.append(model)

del state_dict

In [59]:
sz = 256
bs = 1
folds = 4
tsfm = data_transform()
tsfm = None
dataset = PandaPatchDataset(csv_file, data_dir, sz, transform=tsfm, N = 36, rand = False)
crossValData = crossValDataloader(csv_file, dataset, bs)

In [60]:
image_features = {}
for fold in [0,1,2,3]:
    ## Model inference
    _, dataloader0 = crossValData(fold)
    model = models[0]
    with torch.no_grad():
        for idx, data in enumerate(tqdm(dataloader0)):
            img, name, provider, grade = data['img'], data['name'], data['datacenter'], data['isup_grade']
            img = img.cuda().float()
            features = model(img)['out']
            image_features[name[0]] = features.cpu().numpy()
#             if idx == 5:
#                 break

Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7f36e5868b00>>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 961, in __del__
    self._shutdown_workers()
  File "/opt/conda/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 941, in _shutdown_workers
    w.join()
  File "/opt/conda/lib/python3.6/multiprocessing/process.py", line 122, in join
    assert self._parent_pid == os.getpid(), 'can only join a child process'
AssertionError: can only join a child process
Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7f36e5868b00>>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 961, in __del__
    self._shutdown_workers()
  File "/opt/c

In [61]:
with open('./input/csv_pkl_files/extracted_image_features.pkl', 'wb') as handle:
    pickle.dump(image_features, handle, protocol=pickle.HIGHEST_PROTOCOL)